# Log Birthdays

In [1]:
from datetime import datetime, timedelta
import pandas as pd

df_log_birth = pd.DataFrame()

birth_date = datetime(year=1990, month=12, day=20, hour=10, minute=42)

for exp in range(11):
    # compute 10 ^ exp seconds after birth_date
    offset = timedelta(seconds=(10 ** exp))
    
    # compute moment of the log birthday
    bday = birth_date + offset
    
    # build a row of log birthday party
    s_bday = pd.Series({'bday': f'10^{exp} sec',
              'year': bday.year,
              'month': bday.month,
              'day': bday.day,
              'hour': bday.hour,
              'minute': bday.minute,
              'sec': bday.second,
              'read_day': bday.strftime('%B-%d-%Y %H:%M %p')})
    
    # store it
    df_log_birth = pd.concat([df_log_birth, s_bday.to_frame().T], ignore_index=True)

df_log_birth

,bday,year,month,day,hour,minute,sec,read_day
0,10^0 sec,1990,12,20,10,42,1,December-20-1990 10:42 AM
1,10^1 sec,1990,12,20,10,42,10,December-20-1990 10:42 AM
2,10^2 sec,1990,12,20,10,43,40,December-20-1990 10:43 AM
3,10^3 sec,1990,12,20,10,58,40,December-20-1990 10:58 AM
4,10^4 sec,1990,12,20,13,28,40,December-20-1990 13:28 PM
5,10^5 sec,1990,12,21,14,28,40,December-21-1990 14:28 PM
6,10^6 sec,1991,1,1,0,28,40,January-01-1991 00:28 AM
7,10^7 sec,1991,4,15,4,28,40,April-15-1991 04:28 AM
8,10^8 sec,1994,2,19,20,28,40,February-19-1994 20:28 PM
9,10^9 sec,2022,8,28,12,28,40,August-28-2022 12:28 PM


# Penguin Dictionary

In [2]:
grn_dict = {'height': [2, 3, 2],
            'weight': [4, 6, 5]}
blu_dict = {'height': [4, 6, 5],
            'weight': [2, 3, 2]}
red_dict = {'height': [10, 7, 8],
            'weight': [11, 6, 5]}
peng_data = [grn_dict,
             blu_dict,
             red_dict]

# Getting Forecast from OpenWeather


In [3]:
import requests
import json
import pandas as pd
from datetime import datetime


def get_forecast(lat, lon, units='imperial', api_key='d36fa352ac73226b30772f64675f41bb'):
    """ creates a function that gets the hourly forecast (for the next two days) for a given lat and lon
    
    Args:
        lat (float): latitude
        lon (float): longitude
        units (str): units
        api_key (str): key for accessing API
        
    Returns:
        df_hourly (data frame): data frame with rows corresponding to hours and columns various features
    
    """
    # build url
    url = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&appid={api_key}&units={units}'

    # get url response as a string
    url_text = requests.get(url).text

    # convert from json to a nested dict
    weather_dict = json.loads(url_text)

    # build dataframe from nested dict
    df_hourly = pd.DataFrame()
    for hour_dict in weather_dict['hourly']:
        # we add a 'datetime' feature from the unix time (easier to use)
        hour_dict['datetime'] = datetime.fromtimestamp(hour_dict['dt'])

        # build a single row of dataframe (weather prediction for some hour)
        s_hour = pd.Series(hour_dict)

        # collect forecasts in a dataframe
        df_hourly = df_hourly.append(s_hour, ignore_index=True)

    return df_hourly